# Static Features

This notebook will read some function graphs extracted with the idapython plugin, but this time the graph of each function contains information about the code. It's a set of features, the code features, that comprehend counts of lines of code, registers, mem addresses, function calls, and two different outputs of the code as a document to be later used with a bag of words embedding.

Tasks:
-------

1. modify readGraph() function
2. numbers of (reg, memaddr, xrefs, ...)
3. doc and doc2 with code
4. verification of the function graph and the links vs the original function
5. verification that the nubers associated to registers are consistent across all imported functions
6. create the BOW models:
    * a specific one with manually selected vocabulary
    * a more generic one using TF-IDF to filter out non intersting words from the total vocabulary


## 1. Implement the reading of these new features on readGraph()


## 2. Import dataset with code features



In [177]:
%load_ext autoreload
%autoreload 2

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir+'/graph_classification') 

import importlib
import time
import pickle
import traceback
import random
import os
import re
import sys
import json
from pprint import pprint
import numpy as np
from numpy.random import choice
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import torch
from torch_geometric.data import Dataset
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

from TFM_function_renaming_dataset_creation import *
from TFM_function_renaming_dataset_creation import FunctionsDataset
from TFM_graph_classification import *
from TFM_graph_classification_models import *
from TFM_graph_classification_baseline_models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
dataset = FunctionsDataset(root='./tmp/for_code_features')
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_features)
print(len(dataset.problematic_files))
#pprint(dataset.problematic_files)
print(dataset[0].filename)
print(dataset[0].code_feats['document'])
print(dataset[1].filename)
print(dataset[1].code_feats)

before super init
['BaseThreadInitThunk', 'InterlockedPushListSList', 'AcquireSRWLockExclusive', 'AcquireSRWLockShared', 'AcquireStateLock', 'ActivateActCtx', 'ActivateActCtxWorker', 'AddAtomA', 'AddAtomW', 'AddConsoleAliasA']
['AAA', 'AAD', 'AAM', 'AAS', 'ADC', 'ADCX', 'ADD', 'ADDPD', 'ADDPS', 'ADDSD']
['instr', 'register', 'immediate', 'memory', 'displacement', 'phrase', 'func', 'unknown']
processed_path tmp/for_code_features/processed
Processing...
 FunctionsDataset: process()
['BaseThreadInitThunk', 'InterlockedPushListSList', 'AcquireSRWLockExclusive', 'AcquireSRWLockShared', 'AcquireStateLock', 'ActivateActCtx', 'ActivateActCtxWorker', 'AddAtomA', 'AddAtomW', 'AddConsoleAliasA']
['AAA', 'AAD', 'AAM', 'AAS', 'ADC', 'ADCX', 'ADD', 'ADDPD', 'ADDPS', 'ADDSD']
['instr', 'register', 'immediate', 'memory', 'displacement', 'phrase', 'func', 'unknown']
processed_path tmp/for_code_features/processed
self.processed_paths []
self.root tmp/for_code_features
self.raw_dir tmp/for_code_features/

Traceback (most recent call last):
  File "/media/disk/home/pau/Projectes/GNN-MThesis/src/function_renaming/TFM_function_renaming_dataset_creation.py", line 403, in parseGraph
    g = readGraph('',filename)
  File "/media/disk/home/pau/Projectes/GNN-MThesis/src/function_renaming/TFM_function_renaming_dataset_creation.py", line 73, in readGraph
    for line in f.readlines():
  File "/home/pau/.pyenv/versions/3.6.7/lib/python3.6/codecs.py", line 321, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x88 in position 273: invalid start byte


tmp/for_code_features/raw/graphs01/Network/malware/sub_457520_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/SleepLoop_45F2A0_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_454C40_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_454170_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_45D850_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_455B60_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_455980_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/roadToDecryptionRoutine_45FC60_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/WinHttp_and_Decryption_45FF90_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_45F370_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_455360_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_454250_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/IATrebuild2_WinHTTP_45AED0_edges.txt
tmp

Traceback (most recent call last):
  File "/media/disk/home/pau/Projectes/GNN-MThesis/src/function_renaming/TFM_function_renaming_dataset_creation.py", line 403, in parseGraph
    g = readGraph('',filename)
  File "/media/disk/home/pau/Projectes/GNN-MThesis/src/function_renaming/TFM_function_renaming_dataset_creation.py", line 81, in readGraph
    attr_dict = json.loads(tokens)
  File "/home/pau/.pyenv/versions/3.6.7/lib/python3.6/json/__init__.py", line 354, in loads
    return _default_decoder.decode(s)
  File "/home/pau/.pyenv/versions/3.6.7/lib/python3.6/json/decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/home/pau/.pyenv/versions/3.6.7/lib/python3.6/json/decoder.py", line 355, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Invalid \escape: line 1 column 149 (char 148)


tmp/for_code_features/raw/graphs01/Network/malware/sub_466A10_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_457670_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_466EB2_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/__SEH_prolog_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/ThreadStart_4604D0_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_466CA1_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_451150_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_45E530_edges.txt


...

tmp/for_code_features/raw/graphs01/Network/malware/sub_460D80_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_451DE0_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_456A00_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_466EBE_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_457160_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_466490_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_460B05_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_466590_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/unknown_GatherHostnamesAndIPaddress_45AD90_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/RevertToSelf_and_saveto_dw46b108_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/Memmove_ASCII_hostsnames_45AC70_edges.txt


Traceback (most recent call last):
  File "/media/disk/home/pau/Projectes/GNN-MThesis/src/function_renaming/TFM_function_renaming_dataset_creation.py", line 403, in parseGraph
    g = readGraph('',filename)
  File "/media/disk/home/pau/Projectes/GNN-MThesis/src/function_renaming/TFM_function_renaming_dataset_creation.py", line 73, in readGraph
    for line in f.readlines():
  File "/home/pau/.pyenv/versions/3.6.7/lib/python3.6/codecs.py", line 321, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa5 in position 7524: invalid start byte


tmp/for_code_features/raw/graphs01/Network/malware/sub_4552E0_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_454700_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_452740_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_45D270_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/j_ObtainUserAgentString_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/GetTickCount_wrapper_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_466EAC_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_466990_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/WINHTTP_452F00_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_45BE90_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_456D70_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/ThreadNew_460870_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_454550_edges.txt
tmp/for_code_features/raw/graph

Traceback (most recent call last):
  File "/media/disk/home/pau/Projectes/GNN-MThesis/src/function_renaming/TFM_function_renaming_dataset_creation.py", line 403, in parseGraph
    g = readGraph('',filename)
  File "/media/disk/home/pau/Projectes/GNN-MThesis/src/function_renaming/TFM_function_renaming_dataset_creation.py", line 81, in readGraph
    attr_dict = json.loads(tokens)
  File "/home/pau/.pyenv/versions/3.6.7/lib/python3.6/json/__init__.py", line 354, in loads
    return _default_decoder.decode(s)
  File "/home/pau/.pyenv/versions/3.6.7/lib/python3.6/json/decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/home/pau/.pyenv/versions/3.6.7/lib/python3.6/json/decoder.py", line 355, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Expecting ',' delimiter: line 1 column 142 (char 141)


tmp/for_code_features/raw/graphs01/Network/malware/sub_454D00_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_454450_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_4511A0_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_4522E0_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_455A70_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_452440_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_4518F0_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/j_CreateThread_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/HOSTNAMES_Write_Hostnames_to_Stack_4581C0_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_4515B0_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/unknow_getTickCount_interlockedExchange_458070_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/sub_452770_edges.txt
tmp/for_code_features/raw/graphs01/Network/malware/INI_GLOBAL_VAR

In [6]:
dir(dataset[0])

['__call__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply',
 'cat_dim',
 'clone',
 'code_feats',
 'contains_isolated_nodes',
 'contains_self_loops',
 'contiguous',
 'edge_attr',
 'edge_index',
 'filename',
 'from_dict',
 'is_coalesced',
 'is_directed',
 'is_undirected',
 'keys',
 'num_edges',
 'num_features',
 'num_nodes',
 'pos',
 'to',
 'x',
 'x_topo_feats',
 'x_topo_times',
 'y']

In [2]:
dataset = FunctionsDataset(root='./tmp/for_code_features')
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_features)
print(len(dataset.problematic_files))
#pprint(dataset.problematic_files)
print(dataset[0].filename)
print(dataset[1].filename)

processed_path tmp/rethinking_static_features/processed
 end of __init__()
541
3
4
0
tmp/rethinking_static_features/raw/graphs01/Network/ApacheCore_i64/sub_6FF81010_edges.txt
tmp/rethinking_static_features/raw/graphs01/Disk/MEBackup_i64/sub_405300_edges.txt


## 3. Verification
1. verify that the other code features are correct and doc and doc2 are correct too
    - GT print assembly of a selected function
    - GT printscreen ida blockview 
    - print document, and document2 and compare
    - print list of registers and compare
    - print list of funcs  and compare
2. verification of the function graph and the links vs the original function
    - print a graph with the node attributes (content) and compare
3. verification that the numbers(ids) associated to registers are consistent across all imported functions


### 3.1 Automated verification of Call_Decryption_Routine function

In [181]:
%load_ext autoreload
%autoreload 2

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir+'/graph_classification') 

import importlib
import time
import pickle
import traceback
import random
import os
import re
import sys
import json
from pprint import pprint
import numpy as np
from numpy.random import choice
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import torch
from torch_geometric.data import Dataset
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

from TFM_function_renaming_dataset_creation import *
from TFM_function_renaming_dataset_creation import FunctionsDataset
from TFM_graph_classification import *
from TFM_graph_classification_models import *
from TFM_graph_classification_baseline_models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [182]:
test_code_features(dataset[146].code_feats,
                   'verification/Call_Decryption_Routine_45E320.txt',
                 'tmp/for_code_features/raw/graphs01/Network/malware/Call_Decryption_Routine_45E320_nodes.txt',
                   'tmp/for_code_features/raw/graphs01/Network/malware/Call_Decryption_Routine_45E320_edges.txt')

0x45e320  Call_Decryption_Routine_45E320 proc near 
contents is not equal:  Call_Decryption_Routine_45E320 proc near |push ebp
0x45e320  
contents is not equal:  |push ebp
0x45e320  var_AC = dword ptr -0ACh 
contents is not equal:  var_AC = dword ptr -0ACh |push ebp
0x45e320  var_A8 = dword ptr -0A8h 
contents is not equal:  var_A8 = dword ptr -0A8h |push ebp
0x45e320  var_A4 = dword ptr -0A4h 
contents is not equal:  var_A4 = dword ptr -0A4h |push ebp
0x45e320  var_A0 = dword ptr -0A0h 
contents is not equal:  var_A0 = dword ptr -0A0h |push ebp
0x45e320  var_9C = dword ptr -9Ch 
contents is not equal:  var_9C = dword ptr -9Ch |push ebp
0x45e320  var_98 = dword ptr -98h 
contents is not equal:  var_98 = dword ptr -98h |push ebp
0x45e320  var_94 = dword ptr -94h 
contents is not equal:  var_94 = dword ptr -94h |push ebp
0x45e320  var_90 = dword ptr -90h 
contents is not equal:  var_90 = dword ptr -90h |push ebp
0x45e320  var_8C = dword ptr -8Ch 
contents is not equal:  var_8C = dword pt

found lea     eax, [ebp+var_88] 
 found  0045E3EF  lea eax, [ebp+var_88] 
found push    eax 
 found  0045E3F5  push eax 
found lea     ecx, [ebp+var_3C] 
 found  0045E3F6  lea ecx, [ebp+var_3C] 
found push    ecx 
 found  0045E3F9  push ecx 
found call    sprintf 
 found  0045E3FA  call sprintf 
found add     esp, 0Ch 
 found  0045E400  add esp, 0Ch 
found xor     esi, esi 
 found  0045E403  xor esi, esi 
found mov     [ebp+var_94], esi 
 found  0045E405  mov [ebp+var_94], esi 
found cmp     esi, 3 
 found  0045E40B  cmp esi, 3 
found jge     loc_45E4FD 
 found  0045E40E  jge loc_45E4FD 
found test    esi, esi 
 found  0045E414  test esi, esi 
found jnz     short loc_45E41F 
 found  0045E416  jnz short loc_45E41F 
found mov     ecx, 80000002h 
 found  0045E418  mov ecx, 80000002h 
found jmp     short loc_45E432 
 found  0045E41D  jmp short loc_45E432 
found mov     ecx, esi 
 found  0045E41F  mov ecx, esi 
found dec     ecx 
 found  0045E421  dec ecx 
found neg     ecx 
 found  0045E42

In [45]:
!pwd
!ls tmp/for_code_

/media/disk/home/pau/Projectes/GNN-MThesis/src/function_renaming


#### Call_DecryptionRoutine_45e320 Assembly listing

![disassembler listing](./verification/CallDecryption01.PNG)
![disassembler listing2](./verification/CallDecryption02.PNG)
![disassembler listing3](./verification/CallDecryption03.PNG)
![disassembler listing4](./verification/CallDecryption04.PNG)

`
UPX0:0045E320 Call_Decryption_Routine_45E320 proc near ; CODE XREF: sub_45E760+97p
UPX0:0045E320                                         ; INI_GLOBAL_VARS_45ED00+2Ep ...
UPX0:0045E320
UPX0:0045E320 var_AC          = dword ptr -0ACh
UPX0:0045E320 var_A8          = dword ptr -0A8h
UPX0:0045E320 var_A4          = dword ptr -0A4h
UPX0:0045E320 var_A0          = dword ptr -0A0h
UPX0:0045E320 var_9C          = dword ptr -9Ch
UPX0:0045E320 var_98          = dword ptr -98h
UPX0:0045E320 var_94          = dword ptr -94h
UPX0:0045E320 var_90          = dword ptr -90h
UPX0:0045E320 var_8C          = dword ptr -8Ch
UPX0:0045E320 var_88          = byte ptr -88h
UPX0:0045E320 var_84          = dword ptr -84h
UPX0:0045E320 var_80          = dword ptr -80h
UPX0:0045E320 var_7C          = dword ptr -7Ch
UPX0:0045E320 var_78          = byte ptr -78h
UPX0:0045E320 var_3C          = byte ptr -3Ch
UPX0:0045E320 var_1C          = dword ptr -1Ch
UPX0:0045E320 ms_exc          = CPPEH_RECORD ptr -18h
UPX0:0045E320 arg_0           = dword ptr  8
UPX0:0045E320 arg_4           = dword ptr  0Ch
UPX0:0045E320 arg_8           = dword ptr  10h
UPX0:0045E320
UPX0:0045E320                 push    ebp
UPX0:0045E321                 mov     ebp, esp
UPX0:0045E323                 push    0FFFFFFFFh
UPX0:0045E325                 push    offset stru_4655A8
UPX0:0045E32A                 push    offset Strange_Bottleneck_sub_460A0C
UPX0:0045E32F                 mov     eax, large fs:0
UPX0:0045E335                 push    eax
UPX0:0045E336                 mov     large fs:0, esp
UPX0:0045E33D                 sub     esp, 9Ch
UPX0:0045E343                 mov     eax, ds:Check_Var_469864
UPX0:0045E348                 xor     eax, ebp
UPX0:0045E34A                 mov     [ebp+var_1C], eax
UPX0:0045E34D                 push    ebx
UPX0:0045E34E                 push    esi
UPX0:0045E34F                 push    edi
UPX0:0045E350                 mov     [ebp+ms_exc.old_esp], esp
UPX0:0045E353                 mov     [ebp+var_9C], edx
UPX0:0045E359                 mov     [ebp+var_90], ecx
UPX0:0045E35F                 mov     esi, [ebp+arg_0]
UPX0:0045E362                 mov     [ebp+var_80], esi
UPX0:0045E365                 xor     eax, eax
UPX0:0045E367                 mov     [ebp+ms_exc.registration.TryLevel], eax
UPX0:0045E36A                 mov     ecx, 6F58h
UPX0:0045E36F                 mov     [ebp+var_A0], ecx
UPX0:0045E375
UPX0:0045E375 loc_45E375:                             ; CODE XREF: Call_Decryption_Routine_45E320+7Bj
UPX0:0045E375                 mov     [ebp+var_A8], eax
UPX0:0045E37B                 cmp     eax, 3Ch
UPX0:0045E37E                 jge     short loc_45E39D
UPX0:0045E380                 add     ecx, 0D7D9h
UPX0:0045E386                 mov     [ebp+var_A0], ecx
UPX0:0045E38C                 xor     ebx, ebx
UPX0:0045E38E                 mov     bl, ADD_deobfuscation_461E60[eax]
UPX0:0045E394                 add     ebx, ecx
UPX0:0045E396                 mov     [ebp+eax+var_78], bl
UPX0:0045E39A                 inc     eax
UPX0:0045E39B                 jmp     short loc_45E375
UPX0:0045E39D ; ---------------------------------------------------------------------------
UPX0:0045E39D
UPX0:0045E39D loc_45E39D:                             ; CODE XREF: Call_Decryption_Routine_45E320+5Ej
UPX0:0045E39D                 mov     byte ptr [esi], 0
UPX0:0045E3A0                 mov     ecx, 0DFEB5B94h
UPX0:0045E3A5                 mov     [ebp+var_A4], ecx
UPX0:0045E3AB                 xor     eax, eax
UPX0:0045E3AD                 mov     [ebp+var_8C], eax
UPX0:0045E3B3                 mov     edi, 3
UPX0:0045E3B8                 mov     [ebp+var_AC], edi
UPX0:0045E3BE                 mov     edi, edi
UPX0:0045E3C0
UPX0:0045E3C0 loc_45E3C0:                             ; CODE XREF: Call_Decryption_Routine_45E320+CCj
UPX0:0045E3C0                 cmp     ax, di
UPX0:0045E3C3                 jnb     short loc_45E3EE
UPX0:0045E3C5                 sub     ecx, 125A0CA9h
UPX0:0045E3CB                 mov     [ebp+var_A4], ecx
UPX0:0045E3D1                 movzx   esi, ax
UPX0:0045E3D4                 xor     ebx, ebx
UPX0:0045E3D6                 mov     bl, byte_461E5D[esi]
UPX0:0045E3DC                 sub     ebx, ecx
UPX0:0045E3DE                 mov     [ebp+esi+var_88], bl
UPX0:0045E3E5                 inc     eax
UPX0:0045E3E6                 mov     [ebp+var_8C], eax
UPX0:0045E3EC                 jmp     short loc_45E3C0
UPX0:0045E3EE ; ---------------------------------------------------------------------------
UPX0:0045E3EE
UPX0:0045E3EE loc_45E3EE:                             ; CODE XREF: Call_Decryption_Routine_45E320+A3j
UPX0:0045E3EE                 push    edx
UPX0:0045E3EF                 lea     eax, [ebp+var_88]
UPX0:0045E3F5                 push    eax
UPX0:0045E3F6                 lea     ecx, [ebp+var_3C]
UPX0:0045E3F9                 push    ecx
UPX0:0045E3FA                 call    sprintf
UPX0:0045E400                 add     esp, 0Ch
UPX0:0045E403                 xor     esi, esi
UPX0:0045E405
UPX0:0045E405 loc_45E405:                             ; CODE XREF: Call_Decryption_Routine_45E320+1CFj
UPX0:0045E405                 mov     [ebp+var_94], esi
UPX0:0045E40B                 cmp     esi, 3
UPX0:0045E40E                 jge     loc_45E4FD
UPX0:0045E414                 test    esi, esi
UPX0:0045E416                 jnz     short loc_45E41F
UPX0:0045E418                 mov     ecx, 80000002h
UPX0:0045E41D                 jmp     short loc_45E432
UPX0:0045E41F ; ---------------------------------------------------------------------------
UPX0:0045E41F
UPX0:0045E41F loc_45E41F:                             ; CODE XREF: Call_Decryption_Routine_45E320+F6j
UPX0:0045E41F                 mov     ecx, esi
UPX0:0045E421                 dec     ecx
UPX0:0045E422                 neg     ecx
UPX0:0045E424                 sbb     ecx, ecx
UPX0:0045E426                 and     ecx, 7FFFFFFEh
UPX0:0045E42C                 add     ecx, 80000001h
UPX0:0045E432
UPX0:0045E432 loc_45E432:                             ; CODE XREF: Call_Decryption_Routine_45E320+FDj
UPX0:0045E432                 lea     edx, [ebp+var_98]
UPX0:0045E438                 push    edx
UPX0:0045E439                 lea     eax, [ebp+var_84]
UPX0:0045E43F                 push    eax
UPX0:0045E440                 lea     edx, [ebp+var_3C]
UPX0:0045E443                 push    edx
UPX0:0045E444                 lea     edx, [ebp+var_78]
UPX0:0045E447                 call    sub_45DA00
UPX0:0045E44C                 test    eax, eax
UPX0:0045E44E                 jz      loc_45E4EE
UPX0:0045E454                 xor     ebx, ebx
UPX0:0045E456                 mov     [ebp+var_7C], ebx
UPX0:0045E459                 mov     esi, [ebp+var_84]
UPX0:0045E45F                 test    esi, esi
UPX0:0045E461                 jz      short loc_45E495
UPX0:0045E463                 mov     ecx, [ebp+var_98]
UPX0:0045E469                 test    ecx, ecx
UPX0:0045E46B                 jz      short loc_45E495
UPX0:0045E46D                 mov     eax, [ebp+arg_4]
UPX0:0045E470                 sub     eax, [ebp+arg_8]
UPX0:0045E473                 cmp     eax, ecx
UPX0:0045E475                 jl      short loc_45E479
UPX0:0045E477                 mov     eax, ecx
UPX0:0045E479
UPX0:0045E479 loc_45E479:                             ; CODE XREF: Call_Decryption_Routine_45E320+155j
UPX0:0045E479                 mov     ebx, eax
UPX0:0045E47B                 mov     [ebp+var_7C], ebx
UPX0:0045E47E                 mov     ecx, ebx
UPX0:0045E480                 mov     edi, [ebp+var_80]
UPX0:0045E483                 shr     ecx, 2
UPX0:0045E486                 rep movsd
UPX0:0045E488                 mov     ecx, eax
UPX0:0045E48A                 and     ecx, 3
UPX0:0045E48D                 rep movsb
UPX0:0045E48F                 mov     esi, [ebp+var_84]
UPX0:0045E495
UPX0:0045E495 loc_45E495:                             ; CODE XREF: Call_Decryption_Routine_45E320+141j
UPX0:0045E495                                         ; Call_Decryption_Routine_45E320+14Bj
UPX0:0045E495                 mov     eax, [ebp+var_9C]
UPX0:0045E49B                 imul    eax, 64h
UPX0:0045E49E                 add     eax, 1000h
UPX0:0045E4A3                 push    eax
UPX0:0045E4A4                 push    ebx
UPX0:0045E4A5                 mov     ecx, [ebp+var_80]
UPX0:0045E4A8                 push    ecx             ; encrypted buffer
UPX0:0045E4A9                 mov     edx, 10h
UPX0:0045E4AE                 mov     ecx, [ebp+var_90]
UPX0:0045E4B4                 call    DecryptionRoutine01_451270
UPX0:0045E4B9                 mov     eax, [ebp+var_80]
UPX0:0045E4BC                 mov     byte ptr [eax+ebx], 0
UPX0:0045E4C0                 inc     ebx
UPX0:0045E4C1                 mov     [ebp+var_7C], ebx
UPX0:0045E4C4                 cmp     [ebp+arg_8], 2
UPX0:0045E4C8                 jnz     short loc_45E4D2
UPX0:0045E4CA                 mov     byte ptr [eax+ebx], 0
UPX0:0045E4CE                 inc     ebx
UPX0:0045E4CF                 mov     [ebp+var_7C], ebx
UPX0:0045E4D2
UPX0:0045E4D2 loc_45E4D2:                             ; CODE XREF: Call_Decryption_Routine_45E320+1A8j
UPX0:0045E4D2                 test    esi, esi
UPX0:0045E4D4                 jz      short loc_45E4E0
UPX0:0045E4D6                 push    esi
UPX0:0045E4D7                 call    free
UPX0:0045E4DD                 add     esp, 4
UPX0:0045E4E0
UPX0:0045E4E0 loc_45E4E0:                             ; CODE XREF: Call_Decryption_Routine_45E320+1B4j
UPX0:0045E4E0                 mov     [ebp+ms_exc.registration.TryLevel], 0FFFFFFFFh
UPX0:0045E4E7                 mov     eax, 1
UPX0:0045E4EC                 jmp     short loc_45E506
UPX0:0045E4EE ; ---------------------------------------------------------------------------
UPX0:0045E4EE
UPX0:0045E4EE loc_45E4EE:                             ; CODE XREF: Call_Decryption_Routine_45E320+12Ej
UPX0:0045E4EE                 inc     esi
UPX0:0045E4EF                 jmp     loc_45E405
UPX0:0045E4F4 ; ---------------------------------------------------------------------------
UPX0:0045E4F4
UPX0:0045E4F4 loc_45E4F4:                             ; DATA XREF: UPX1:stru_4655A8o
UPX0:0045E4F4                 mov     eax, 1          ; Exception filter 0 for function 45E320
UPX0:0045E4F9                 retn
UPX0:0045E4FA ; ---------------------------------------------------------------------------
UPX0:0045E4FA
UPX0:0045E4FA loc_45E4FA:                             ; DATA XREF: UPX1:stru_4655A8o
UPX0:0045E4FA                 mov     esp, [ebp+ms_exc.old_esp] ; Exception handler 0 for function 45E320
UPX0:0045E4FD
UPX0:0045E4FD loc_45E4FD:                             ; CODE XREF: Call_Decryption_Routine_45E320+EEj
UPX0:0045E4FD                 mov     [ebp+ms_exc.registration.TryLevel], 0FFFFFFFFh
UPX0:0045E504                 xor     eax, eax
UPX0:0045E506
UPX0:0045E506 loc_45E506:                             ; CODE XREF: Call_Decryption_Routine_45E320+1CCj
UPX0:0045E506                 mov     ecx, [ebp+ms_exc.registration.Next]
UPX0:0045E509                 mov     large fs:0, ecx
UPX0:0045E510                 pop     edi
UPX0:0045E511                 pop     esi
UPX0:0045E512                 pop     ebx
UPX0:0045E513                 mov     ecx, [ebp+var_1C]
UPX0:0045E516                 xor     ecx, ebp
UPX0:0045E518                 call    Check_Global_Var_460B15
UPX0:0045E51D                 mov     esp, ebp
UPX0:0045E51F                 pop     ebp
UPX0:0045E520                 retn    0Ch
UPX0:0045E520 Call_Decryption_Routine_45E320 endp
UPX0:0045E520%             
`

In [24]:
for i in range(len(dataset)):
   # print(dataset[i].filename,i)
    if dataset[i].filename.find("Call_Decryption")>-1:
        print(dataset[i].filename,i)
    

tmp/for_code_features/raw/graphs01/Network/malware/Call_Decryption_Routine_45E320_edges.txt 146


In [35]:
#dir(dataset[i])
print(dataset[i].code_feats.keys())
func_feats = dataset[i].code_feats
print('nregs',func_feats['nregs'])
print('ninstrs',func_feats['ninstrs'])
print('ndispls',func_feats['ndispls'])
print('nimms',func_feats['nimms'])
print('nmaddrs',func_fbeats['nmaddrs'])
print('num_funcs',func_feats['num_funcs'])
print('list_regs',func_feats['list_regs'])
print('list_funcs',func_feats['list_funcs'])

dict_keys(['nregs', 'ninstrs', 'ndispls', 'nimms', 'nmaddrs', 'num_funcs', 'document', 'document_simplified', 'list_regs', 'list_funcs'])
nregs 12
ninstrs 108
ndispls 16
nimms 12
nmaddrs 4
num_funcs 2
list_regs ['[ebp+var_6C]', '[ebp+var_74]', '[ebp+var_6D]', '[ebp+var_78]', '[ebp+var_7C]', '[ebp+var_1C]', 'al', 'dl', 'bl', '[ebp+var_65]', '[ebp+esi+var_64]', '[eax]', '[ebp+ms_exc.registration.TryLevel]', '[ebp+ms_exc.registration.Next]', 'esp', 'ebp', 'esi', 'edi', 'eax', 'ecx', 'edx', 'ebx', 'ds:byte_467D74[esi]', '[ebp+esi+var_28]', 'ds:byte_467D39[esi]', '[ebp+var_84]', '[ebp+var_80]', '[ebp+ms_exc.old_esp]']
list_funcs ['sub_45DA00', 'Check_Global_Var_460B15']


In [36]:
print('document',func_feats['document'])


document mov     [ebp+var_65], al 
push    ecx 
mov     ecx, 80000002h 
mov     [ebp+var_84], ecx 
xor     al, al 
mov     [ebp+var_1C], eax 
push    edi 
push    ebx 
cmp     [ebp+var_74], 4 
mov     eax, large fs:0 
test    eax, eax 
push    esi 
mov     [ebp+var_84], ecx 
retn 
jz      short loc_45DD79 
mov     [ebp+var_80], edi 
or      edi, 0FFFFFFFFh 
mov     [ebp+ms_exc.registration.TryLevel], ebx 
xor     ebx, ebx 
lea     edx, [ebp+var_74] 
movsx   ecx, al 
jmp     short loc_45DD14 
mov     [ebp+ecx+var_64], bl 
xor     ebx, ecx 
mov     ecx, 63278259h 
mov     [ebp+eax+var_28], bl 
or      eax, 0FFFFFFFFh 
mov     [ebp+var_74], ebx 
mov     esp, [ebp+ms_exc.old_esp]; Exception handler 0 for function 45DC80 
mov     eax, 38A6A03Ch 
mov     [ebp+var_6C], ebx 
lea     eax, [ebp+var_6C] 
mov     dl, ds:byte_467D74[esi] 
xor     edx, edx 
push    offset stru_465578 
push    edx 
push    0FFFFFFFFh 
push    ebp 
mov     ebp, esp 
add     edx, eax 
lea     ecx, [ebp+var_28] 
push   

In [37]:
print('document_simplified',func_feats['document_simplified'])

document_simplified mov [ebp+var_65] al 
push ecx 
mov ecx immediate 
mov [ebp+var_84] ecx 
xor al al 
mov [ebp+var_1C] eax 
push edi 
push ebx 
cmp [ebp+var_74] immediate 
mov eax memory 
test eax eax 
push esi 
mov [ebp+var_84] ecx 
retn 
jz 4578681 
mov [ebp+var_80] edi 
or edi immediate 
mov [ebp+ms_exc.registration.TryLevel] ebx 
xor ebx ebx 
lea edx [ebp+var_74] 
movsx ecx al 
jmp 4578580 
mov [ebp+ecx+var_64] bl 
xor ebx ecx 
mov ecx immediate 
mov [ebp+eax+var_28] bl 
or eax immediate 
mov [ebp+var_74] ebx 
mov esp [ebp+ms_exc.old_esp] 
mov eax immediate 
mov [ebp+var_6C] ebx 
lea eax [ebp+var_6C] 
mov dl ds:byte_467D74[esi] 
xor edx edx 
push immediate 
push edx 
push immediate 
push ebp 
mov ebp esp 
add edx eax 
lea ecx [ebp+var_28] 
push eax 
call sub_45DA00 
mov [ebp+esi+var_28] dl 
jmp 4578507 
mov eax immediate 
inc ecx 
mov eax edi 
movzx eax cx 
jmp 4578724 
mov [ebp+var_78] eax 
shr eax immediate 
xor eax edx 
push immediate 
movzx esi cx 
pop esi 
pop edi 
mov ecx [e

In [41]:
# now print nodes so we can check memaddress
!cat tmp/for_code_features/raw/graphs01/Network/malware/Call_Decryption_Routine_45E320_nodes.txt

In [41]:
# now print nodes so we can check memaddress
!cat tmp/for_code_features/raw/graphs01/Network/malware/Call_Decryption_Routine_45E320_edges.txt

### 3.2 Automated verification of GetIPAddress_of_3_Hostnames functions

#### GetIPAddress_of_3_Hostnames_45EE80

![disassembler listing](./verification/GetIPAddress01.PNG)
![disassembler listing2](./verification/GetIPAddress02.PNG)
![disassembler listing3](./verification/GetIPAddress03.PNG)
![disassembler listing4](./verification/GetIPAddress04.PNG)
![disassembler listing5](./verification/GetIPAddress05.PNG)
![disassembler listing6](./verification/GetIPAddress06.PNG)
![disassembler listing7](./verification/GetIPAddress07.PNG)
![disassembler listing8](./verification/GetIPAddress08.PNG)

`
UPX0:0045EE80 GetIPAddress_of_3_Hostnames_45EE80 proc near
UPX0:0045EE80                                         ; CODE XREF: ThreadStart_4604D0+252p
UPX0:0045EE80
UPX0:0045EE80 var_21B4        = dword ptr -21B4h
UPX0:0045EE80 var_21B0        = dword ptr -21B0h
UPX0:0045EE80 var_21AC        = word ptr -21ACh
UPX0:0045EE80 var_21A8        = dword ptr -21A8h
UPX0:0045EE80 var_21A4        = dword ptr -21A4h
UPX0:0045EE80 var_21A0        = dword ptr -21A0h
UPX0:0045EE80 var_219C        = dword ptr -219Ch
UPX0:0045EE80 var_2198        = dword ptr -2198h
UPX0:0045EE80 var_2191        = byte ptr -2191h
UPX0:0045EE80 Counter_2190    = dword ptr -2190h
UPX0:0045EE80 var_218C        = byte ptr -218Ch
UPX0:0045EE80 var_218B        = byte ptr -218Bh
UPX0:0045EE80 var_218A        = byte ptr -218Ah
UPX0:0045EE80 var_2189        = byte ptr -2189h
UPX0:0045EE80 var_2188        = dword ptr -2188h
UPX0:0045EE80 var_2184        = byte ptr -2184h
UPX0:0045EE80 var_1184        = byte ptr -1184h
UPX0:0045EE80 var_hostname    = byte ptr -184h
UPX0:0045EE80 var_84          = byte ptr -84h
UPX0:0045EE80 var_64          = byte ptr -64h
UPX0:0045EE80 var_34          = byte ptr -34h
UPX0:0045EE80 var_28          = byte ptr -28h
UPX0:0045EE80 var_1C          = dword ptr -1Ch
UPX0:0045EE80 ms_exc          = CPPEH_RECORD ptr -18h
UPX0:0045EE80
UPX0:0045EE80                 push    ebp
UPX0:0045EE81                 mov     ebp, esp
UPX0:0045EE83                 push    0FFFFFFFFh
UPX0:0045EE85                 push    offset stru_465608
UPX0:0045EE8A                 push    offset Strange_Bottleneck_sub_460A0C
UPX0:0045EE8F                 mov     eax, large fs:0
UPX0:0045EE95                 push    eax
UPX0:0045EE96                 mov     large fs:0, esp
UPX0:0045EE9D                 sub     esp, 8
UPX0:0045EEA0                 mov     eax, 219Ch
UPX0:0045EEA5                 call    __alloca_probe
UPX0:0045EEAA                 mov     eax, ds:Check_Var_469864
UPX0:0045EEAF                 xor     eax, ebp
UPX0:0045EEB1                 mov     [ebp+var_1C], eax
UPX0:0045EEB4                 push    ebx
UPX0:0045EEB5                 push    esi
UPX0:0045EEB6                 push    edi
UPX0:0045EEB7                 mov     [ebp+ms_exc.old_esp], esp
UPX0:0045EEBA                 mov     [ebp+var_21A4], edx
UPX0:0045EEC0                 xor     edi, edi
UPX0:0045EEC2                 mov     [ebp+ms_exc.registration.TryLevel], edi
UPX0:0045EEC5                 mov     byte ptr [edx], 0
UPX0:0045EEC8                 push    1
UPX0:0045EECA                 push    0FFFh
UPX0:0045EECF                 lea     eax, [ebp+var_2184]
UPX0:0045EED5                 push    eax
UPX0:0045EED6                 mov     edx, 3
UPX0:0045EEDB                 mov     ecx, offset dword_46987C
UPX0:0045EEE0                 call    Call_Decryption_Routine_45E320
UPX0:0045EEE5                 test    eax, eax
UPX0:0045EEE7                 jnz     short loc_45EEEF
UPX0:0045EEE9                 mov     [ebp+var_2184], al
UPX0:0045EEEF
UPX0:0045EEEF loc_45EEEF:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+67j
UPX0:0045EEEF                 push    1
UPX0:0045EEF1                 push    0FFFh
UPX0:0045EEF6                 lea     ecx, [ebp+var_1184]
UPX0:0045EEFC                 push    ecx
UPX0:0045EEFD                 mov     edx, 2
UPX0:0045EF02                 mov     ecx, offset dword_46987C
UPX0:0045EF07                 call    Call_Decryption_Routine_45E320
UPX0:0045EF0C                 test    eax, eax
UPX0:0045EF0E                 jnz     short loc_45EF16
UPX0:0045EF10                 mov     [ebp+var_1184], al
UPX0:0045EF16
UPX0:0045EF16 loc_45EF16:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+8Ej
UPX0:0045EF16                 mov     dl, 2Ch
UPX0:0045EF18                 lea     ecx, [ebp+var_1184]
UPX0:0045EF1E                 call    sub_458130
UPX0:0045EF23                 cmp     eax, 1
UPX0:0045EF26                 jg      short loc_45EF91
UPX0:0045EF28                 mov     eax, 6626h
UPX0:0045EF2D                 mov     [ebp+var_21B0], eax
UPX0:0045EF33                 xor     cl, cl
UPX0:0045EF35                 mov     [ebp+var_218A], cl
UPX0:0045EF3B                 mov     dl, 2Fh
UPX0:0045EF3D                 mov     [ebp+var_2191], dl
UPX0:0045EF43
UPX0:0045EF43 loc_45EF43:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+F4j
UPX0:0045EF43                 cmp     cl, dl
UPX0:0045EF45                 jnb     short loc_45EF76
UPX0:0045EF47                 movzx   eax, ax
UPX0:0045EF4A                 mov     esi, eax
UPX0:0045EF4C                 shr     esi, 8
UPX0:0045EF4F                 shl     eax, 8
UPX0:0045EF52                 xor     eax, esi
UPX0:0045EF54                 mov     [ebp+var_21B0], eax
UPX0:0045EF5A                 movzx   esi, cl
UPX0:0045EF5D                 xor     ebx, ebx
UPX0:0045EF5F                 mov     bl, ds:byte_4650C8[esi*4]
UPX0:0045EF66                 add     ebx, eax
UPX0:0045EF68                 mov     [ebp+esi+var_64], bl
UPX0:0045EF6C                 inc     cl
UPX0:0045EF6E                 mov     [ebp+var_218A], cl
UPX0:0045EF74                 jmp     short loc_45EF43
UPX0:0045EF76 ; ---------------------------------------------------------------------------
UPX0:0045EF76
UPX0:0045EF76 loc_45EF76:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+C5j
UPX0:0045EF76                 movzx   edx, cl
UPX0:0045EF79                 mov     [ebp+edx+var_64], 0
UPX0:0045EF7E                 lea     ecx, [ebp+var_64]
UPX0:0045EF81                 lea     edx, [ebp+var_1184]
UPX0:0045EF87
UPX0:0045EF87 loc_45EF87:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+10Fj
UPX0:0045EF87                 mov     al, [ecx]
UPX0:0045EF89                 mov     [edx], al
UPX0:0045EF8B                 inc     ecx
UPX0:0045EF8C                 inc     edx
UPX0:0045EF8D                 test    al, al
UPX0:0045EF8F                 jnz     short loc_45EF87
UPX0:0045EF91
UPX0:0045EF91 loc_45EF91:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+A6j
UPX0:0045EF91                 mov     [ebp+Counter_2190], edi
UPX0:0045EF97                 jmp     short loc_45EFA0
UPX0:0045EF97 ; ---------------------------------------------------------------------------
UPX0:0045EF99                 align 10h
UPX0:0045EFA0
UPX0:0045EFA0 loc_45EFA0:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+117j
UPX0:0045EFA0                                         ; GetIPAddress_of_3_Hostnames_45EE80+3E4j
UPX0:0045EFA0                 cmp     [ebp+Counter_2190], 3
UPX0:0045EFA7                 jge     loc_45F272
UPX0:0045EFAD                 mov     dl, 2Ch
UPX0:0045EFAF                 lea     ecx, [ebp+var_2184]
UPX0:0045EFB5                 call    sub_458130
UPX0:0045EFBA                 cmp     eax, 1
UPX0:0045EFBD                 jle     short loc_45EFD3
UPX0:0045EFBF                 mov     ecx, 0Ah
UPX0:0045EFC4                 call    unknow_getTickCount_interlockedExchange_458070
UPX0:0045EFC9                 test    eax, eax
UPX0:0045EFCB                 lea     esi, [ebp+var_2184]
UPX0:0045EFD1                 jz      short loc_45EFD9
UPX0:0045EFD3
UPX0:0045EFD3 loc_45EFD3:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+13Dj
UPX0:0045EFD3                 lea     esi, [ebp+var_1184]
UPX0:0045EFD9
UPX0:0045EFD9 loc_45EFD9:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+151j
UPX0:0045EFD9                 mov     [ebp+var_21A0], esi
UPX0:0045EFDF                 mov     dl, 2Ch
UPX0:0045EFE1                 mov     ecx, esi
UPX0:0045EFE3                 call    sub_458130
UPX0:0045EFE8                 cmp     eax, 1
UPX0:0045EFEB                 jle     loc_45F25E
UPX0:0045EFF1                 mov     dl, 2Ch
UPX0:0045EFF3                 mov     ecx, esi
UPX0:0045EFF5                 call    sub_458130
UPX0:0045EFFA                 mov     ecx, eax
UPX0:0045EFFC                 dec     ecx
UPX0:0045EFFD                 call    unknow_getTickCount_interlockedExchange_458070
UPX0:0045F002                 inc     eax
UPX0:0045F003                 mov     [ebp+var_21B4], eax
UPX0:0045F009                 push    0FFh
UPX0:0045F00E                 lea     ecx, [ebp+var_hostname]
UPX0:0045F014                 push    ecx
UPX0:0045F015                 push    eax
UPX0:0045F016                 mov     dl, 2Ch
UPX0:0045F018                 mov     ecx, esi
UPX0:0045F01A                 call    HOSTNAMES_Write_Hostnames_to_Stack_4581C0
UPX0:0045F01F                 cmp     eax, 0FFFFFFFFh
UPX0:0045F022                 jnz     short loc_45F02E
UPX0:0045F024                 xor     al, al
UPX0:0045F026                 mov     [ebp+var_hostname], al
UPX0:0045F02C                 jmp     short loc_45F034
UPX0:0045F02E ; ---------------------------------------------------------------------------
UPX0:0045F02E
UPX0:0045F02E loc_45F02E:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+1A2j
UPX0:0045F02E                 mov     al, [ebp+var_hostname]
UPX0:0045F034
UPX0:0045F034 loc_45F034:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+1ACj
UPX0:0045F034                 test    al, al
UPX0:0045F036                 jz      loc_45F25E
UPX0:0045F03C                 xor     ebx, ebx
UPX0:0045F03E                 mov     [ebp+var_2188], ebx
UPX0:0045F044                 xor     edi, edi
UPX0:0045F046                 mov     [ebp+var_2198], edi
UPX0:0045F04C                 cmp     al, 30h
UPX0:0045F04E                 jl      loc_45F155
UPX0:0045F054                 cmp     al, 39h
UPX0:0045F056                 jg      loc_45F155
UPX0:0045F05C                 lea     edx, [ebp+var_hostname]
UPX0:0045F062                 push    edx
UPX0:0045F063                 call    inet_addr
UPX0:0045F069                 mov     ebx, eax
UPX0:0045F06B                 mov     [ebp+var_21A8], ebx
UPX0:0045F071                 mov     al, 0D9h
UPX0:0045F073                 mov     [ebp+var_218C], al
UPX0:0045F079                 xor     cl, cl
UPX0:0045F07B                 mov     [ebp+var_2189], cl
UPX0:0045F081                 mov     dl, ds:byte_46953C
UPX0:0045F087                 xor     dl, al
UPX0:0045F089                 movzx   dx, dl
UPX0:0045F08D                 mov     [ebp+var_21AC], dx
UPX0:0045F094
UPX0:0045F094 loc_45F094:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+24Cj
UPX0:0045F094                 cmp     cl, byte ptr [ebp+var_21AC]
UPX0:0045F09A                 jge     short loc_45F0CE
UPX0:0045F09C                 movzx   eax, al
UPX0:0045F09F                 mov     edx, eax
UPX0:0045F0A1                 shr     edx, 2
UPX0:0045F0A4                 shl     eax, 6
UPX0:0045F0A7                 or      edx, eax
UPX0:0045F0A9                 xor     eax, eax
UPX0:0045F0AB                 mov     al, dl
UPX0:0045F0AD                 mov     [ebp+var_218C], al
UPX0:0045F0B3                 movsx   esi, cl
UPX0:0045F0B6                 xor     edx, edx
UPX0:0045F0B8                 mov     dl, ds:byte_46953D[esi]
UPX0:0045F0BE                 xor     edx, eax
UPX0:0045F0C0                 mov     [ebp+esi+var_28], dl
UPX0:0045F0C4                 inc     cl
UPX0:0045F0C6                 mov     [ebp+var_2189], cl
UPX0:0045F0CC                 jmp     short loc_45F094
UPX0:0045F0CE ; ---------------------------------------------------------------------------
UPX0:0045F0CE
UPX0:0045F0CE loc_45F0CE:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+21Aj
UPX0:0045F0CE                 movsx   eax, cl
UPX0:0045F0D1                 mov     [ebp+eax+var_28], 0
UPX0:0045F0D6                 mov     ecx, ebx
UPX0:0045F0D8                 shr     ecx, 18h
UPX0:0045F0DB                 push    ecx
UPX0:0045F0DC                 movzx   edx, byte ptr [ebp+var_21A8+2]
UPX0:0045F0E3                 push    edx
UPX0:0045F0E4                 movzx   eax, bh
UPX0:0045F0E7                 push    eax
UPX0:0045F0E8                 mov     ecx, ebx
UPX0:0045F0EA                 and     ecx, 0FFh
UPX0:0045F0F0                 push    ecx
UPX0:0045F0F1                 lea     edx, [ebp+var_28]
UPX0:0045F0F4                 push    edx
UPX0:0045F0F5                 lea     eax, [ebp+var_84]
UPX0:0045F0FB                 push    eax
UPX0:0045F0FC                 call    sprintf
UPX0:0045F102                 add     esp, 18h
UPX0:0045F105                 lea     esi, [ebp+var_84]
UPX0:0045F10B                 lea     eax, [ebp+var_hostname]
UPX0:0045F111
UPX0:0045F111 loc_45F111:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+2AFj
UPX0:0045F111                 mov     dl, [eax]
UPX0:0045F113                 mov     cl, dl
UPX0:0045F115                 cmp     dl, [esi]
UPX0:0045F117                 jnz     short loc_45F135
UPX0:0045F119                 test    cl, cl
UPX0:0045F11B                 jz      short loc_45F131
UPX0:0045F11D                 mov     dl, [eax+1]
UPX0:0045F120                 mov     cl, dl
UPX0:0045F122                 cmp     dl, [esi+1]
UPX0:0045F125                 jnz     short loc_45F135
UPX0:0045F127                 add     eax, 2
UPX0:0045F12A                 add     esi, 2
UPX0:0045F12D                 test    cl, cl
UPX0:0045F12F                 jnz     short loc_45F111
UPX0:0045F131
UPX0:0045F131 loc_45F131:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+29Bj
UPX0:0045F131                 xor     eax, eax
UPX0:0045F133                 jmp     short loc_45F13A
UPX0:0045F135 ; ---------------------------------------------------------------------------
UPX0:0045F135
UPX0:0045F135 loc_45F135:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+297j
UPX0:0045F135                                         ; GetIPAddress_of_3_Hostnames_45EE80+2A5j
UPX0:0045F135                 sbb     eax, eax
UPX0:0045F137                 sbb     eax, 0FFFFFFFFh
UPX0:0045F13A
UPX0:0045F13A loc_45F13A:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+2B3j
UPX0:0045F13A                 test    eax, eax
UPX0:0045F13C                 jnz     short loc_45F14F
UPX0:0045F13E                 mov     [ebp+var_2188], ebx
UPX0:0045F144                 mov     edi, 1
UPX0:0045F149                 mov     [ebp+var_2198], edi
UPX0:0045F14F
UPX0:0045F14F loc_45F14F:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+2BCj
UPX0:0045F14F                 mov     ebx, [ebp+var_2188]
UPX0:0045F155
UPX0:0045F155 loc_45F155:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+1CEj
UPX0:0045F155                                         ; GetIPAddress_of_3_Hostnames_45EE80+1D6j
UPX0:0045F155                 test    edi, edi
UPX0:0045F157                 jnz     short loc_45F196
UPX0:0045F159                 lea     edx, [ebp+var_2188]
UPX0:0045F15F                 lea     ecx, [ebp+var_hostname]
UPX0:0045F165                 call    GetHostbyname_and_more_45A910
UPX0:0045F16A                 test    eax, eax
UPX0:0045F16C                 jnz     short loc_45F183
UPX0:0045F16E                 lea     edx, [ebp+var_2188]
UPX0:0045F174                 lea     ecx, [ebp+var_hostname]
UPX0:0045F17A                 call    DnsQuery_45AB60
UPX0:0045F17F                 test    eax, eax
UPX0:0045F181                 jz      short loc_45F188
UPX0:0045F183
UPX0:0045F183 loc_45F183:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+2ECj
UPX0:0045F183                 mov     eax, 1
UPX0:0045F188
UPX0:0045F188 loc_45F188:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+301j
UPX0:0045F188                 mov     edi, eax
UPX0:0045F18A                 mov     [ebp+var_2198], edi
UPX0:0045F190                 mov     ebx, [ebp+var_2188]
UPX0:0045F196
UPX0:0045F196 loc_45F196:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+2D7j
UPX0:0045F196                 mov     edx, edi
UPX0:0045F198                 neg     edx
UPX0:0045F19A                 sbb     edx, edx
UPX0:0045F19C                 and     edx, ebx
UPX0:0045F19E                 lea     ecx, [ebp+var_hostname]
UPX0:0045F1A4                 call    Memmove_ASCII_hostsnames_45AC70
UPX0:0045F1A9                 test    ebx, ebx
UPX0:0045F1AB                 jz      loc_45F25E
UPX0:0045F1B1                 cmp     ebx, 100007Fh
UPX0:0045F1B7                 jz      loc_45F25E
UPX0:0045F1BD                 cmp     ebx, 0FFFFFFFFh
UPX0:0045F1C0                 jz      loc_45F25E
UPX0:0045F1C6                 lea     ecx, [ebp+var_hostname]
UPX0:0045F1CC                 call    sub_454250
UPX0:0045F1D1                 xor     ebx, eax
UPX0:0045F1D3                 mov     [ebp+var_2188], ebx
UPX0:0045F1D9                 cmp     ebx, 55AA55AAh
UPX0:0045F1DF                 jz      short loc_45F25E
UPX0:0045F1E1                 mov     [ebp+var_219C], 0E42DF193h
UPX0:0045F1EB                 xor     cl, cl
UPX0:0045F1ED                 mov     [ebp+var_218B], cl
UPX0:0045F1F3
UPX0:0045F1F3 loc_45F1F3:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+3AAj
UPX0:0045F1F3                 cmp     cl, 0Ch
UPX0:0045F1F6                 jnb     short loc_45F22C
UPX0:0045F1F8                 mov     esi, [ebp+var_219C]
UPX0:0045F1FE                 sub     esi, 7A3FA98Dh
UPX0:0045F204                 mov     [ebp+var_219C], esi
UPX0:0045F20A                 movzx   eax, cl
UPX0:0045F20D                 xor     edx, edx
UPX0:0045F20F                 mov     dl, ds:byte_465184[eax*4]
UPX0:0045F216                 add     edx, esi
UPX0:0045F218                 mov     [ebp+eax+var_34], dl
UPX0:0045F21C                 inc     cl
UPX0:0045F21E                 mov     [ebp+var_218B], cl
UPX0:0045F224                 mov     ebx, [ebp+var_2188]
UPX0:0045F22A                 jmp     short loc_45F1F3
UPX0:0045F22C ; ---------------------------------------------------------------------------
UPX0:0045F22C
UPX0:0045F22C loc_45F22C:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+376j
UPX0:0045F22C                 mov     eax, ebx
UPX0:0045F22E                 shr     eax, 18h
UPX0:0045F231                 push    eax
UPX0:0045F232                 mov     ecx, [ebp+var_2188]
UPX0:0045F238                 shr     ecx, 10h
UPX0:0045F23B                 movzx   ecx, cl
UPX0:0045F23E                 push    ecx
UPX0:0045F23F                 movzx   edx, bh
UPX0:0045F242                 push    edx
UPX0:0045F243                 and     ebx, 0FFh
UPX0:0045F249                 push    ebx
UPX0:0045F24A                 lea     eax, [ebp+var_34]
UPX0:0045F24D                 push    eax
UPX0:0045F24E                 mov     ecx, [ebp+var_21A4]
UPX0:0045F254                 push    ecx
UPX0:0045F255                 call    sprintf         ; IP address of C&C
UPX0:0045F25B                 add     esp, 18h
UPX0:0045F25E
UPX0:0045F25E loc_45F25E:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+16Bj
UPX0:0045F25E                                         ; GetIPAddress_of_3_Hostnames_45EE80+1B6j ...
UPX0:0045F25E                 inc     [ebp+Counter_2190]
UPX0:0045F264                 jmp     loc_45EFA0
UPX0:0045F269 ; ---------------------------------------------------------------------------
UPX0:0045F269
UPX0:0045F269 loc_45F269:                             ; DATA XREF: UPX1:stru_465608o
UPX0:0045F269                 mov     eax, 1          ; Exception filter 0 for function 45EE80
UPX0:0045F26E                 retn
UPX0:0045F26F ; ---------------------------------------------------------------------------
UPX0:0045F26F
UPX0:0045F26F loc_45F26F:                             ; DATA XREF: UPX1:stru_465608o
UPX0:0045F26F                 mov     esp, [ebp+ms_exc.old_esp] ; Exception handler 0 for function 45EE80
UPX0:0045F272
UPX0:0045F272 loc_45F272:                             ; CODE XREF: GetIPAddress_of_3_Hostnames_45EE80+127j
UPX0:0045F272                 mov     [ebp+ms_exc.registration.TryLevel], 0FFFFFFFFh
UPX0:0045F279                 mov     ecx, [ebp+ms_exc.registration.Next]
UPX0:0045F27C                 mov     large fs:0, ecx
UPX0:0045F283                 pop     edi
UPX0:0045F284                 pop     esi
UPX0:0045F285                 pop     ebx
UPX0:0045F286                 mov     ecx, [ebp+var_1C]
UPX0:0045F289                 xor     ecx, ebp
UPX0:0045F28B                 call    Check_Global_Var_460B15
UPX0:0045F290                 mov     esp, ebp
UPX0:0045F292                 pop     ebp
UPX0:0045F293                 retn
UPX0:0045F293 GetIPAddress_of_3_Hostnames_45EE80 endp
`

In [25]:
for i in range(len(dataset)):
   if dataset[i].filename.find("GetIPAddress")>-1:
        print(dataset[i].filename,i)

tmp/for_code_features/raw/graphs01/Network/malware/GetIPAddress_of_3_Hostnames_45EE80_edges.txt 103


In [35]:
#dir(dataset[i])
print(dataset[i].code_feats.keys())
func_feats = dataset[i].code_feats
print('nregs',func_feats['nregs'])
print('ninstrs',func_feats['ninstrs'])
print('ndispls',func_feats['ndispls'])
print('nimms',func_feats['nimms'])
print('nmaddrs',func_fbeats['nmaddrs'])
print('num_funcs',func_feats['num_funcs'])
print('list_regs',func_feats['list_regs'])
print('list_funcs',func_feats['list_funcs'])

dict_keys(['nregs', 'ninstrs', 'ndispls', 'nimms', 'nmaddrs', 'num_funcs', 'document', 'document_simplified', 'list_regs', 'list_funcs'])
nregs 12
ninstrs 108
ndispls 16
nimms 12
nmaddrs 4
num_funcs 2
list_regs ['[ebp+var_6C]', '[ebp+var_74]', '[ebp+var_6D]', '[ebp+var_78]', '[ebp+var_7C]', '[ebp+var_1C]', 'al', 'dl', 'bl', '[ebp+var_65]', '[ebp+esi+var_64]', '[eax]', '[ebp+ms_exc.registration.TryLevel]', '[ebp+ms_exc.registration.Next]', 'esp', 'ebp', 'esi', 'edi', 'eax', 'ecx', 'edx', 'ebx', 'ds:byte_467D74[esi]', '[ebp+esi+var_28]', 'ds:byte_467D39[esi]', '[ebp+var_84]', '[ebp+var_80]', '[ebp+ms_exc.old_esp]']
list_funcs ['sub_45DA00', 'Check_Global_Var_460B15']


In [36]:
print('document',func_feats['document'])


document mov     [ebp+var_65], al 
push    ecx 
mov     ecx, 80000002h 
mov     [ebp+var_84], ecx 
xor     al, al 
mov     [ebp+var_1C], eax 
push    edi 
push    ebx 
cmp     [ebp+var_74], 4 
mov     eax, large fs:0 
test    eax, eax 
push    esi 
mov     [ebp+var_84], ecx 
retn 
jz      short loc_45DD79 
mov     [ebp+var_80], edi 
or      edi, 0FFFFFFFFh 
mov     [ebp+ms_exc.registration.TryLevel], ebx 
xor     ebx, ebx 
lea     edx, [ebp+var_74] 
movsx   ecx, al 
jmp     short loc_45DD14 
mov     [ebp+ecx+var_64], bl 
xor     ebx, ecx 
mov     ecx, 63278259h 
mov     [ebp+eax+var_28], bl 
or      eax, 0FFFFFFFFh 
mov     [ebp+var_74], ebx 
mov     esp, [ebp+ms_exc.old_esp]; Exception handler 0 for function 45DC80 
mov     eax, 38A6A03Ch 
mov     [ebp+var_6C], ebx 
lea     eax, [ebp+var_6C] 
mov     dl, ds:byte_467D74[esi] 
xor     edx, edx 
push    offset stru_465578 
push    edx 
push    0FFFFFFFFh 
push    ebp 
mov     ebp, esp 
add     edx, eax 
lea     ecx, [ebp+var_28] 
push   

In [37]:
print('document_simplified',func_feats['document_simplified'])

document_simplified mov [ebp+var_65] al 
push ecx 
mov ecx immediate 
mov [ebp+var_84] ecx 
xor al al 
mov [ebp+var_1C] eax 
push edi 
push ebx 
cmp [ebp+var_74] immediate 
mov eax memory 
test eax eax 
push esi 
mov [ebp+var_84] ecx 
retn 
jz 4578681 
mov [ebp+var_80] edi 
or edi immediate 
mov [ebp+ms_exc.registration.TryLevel] ebx 
xor ebx ebx 
lea edx [ebp+var_74] 
movsx ecx al 
jmp 4578580 
mov [ebp+ecx+var_64] bl 
xor ebx ecx 
mov ecx immediate 
mov [ebp+eax+var_28] bl 
or eax immediate 
mov [ebp+var_74] ebx 
mov esp [ebp+ms_exc.old_esp] 
mov eax immediate 
mov [ebp+var_6C] ebx 
lea eax [ebp+var_6C] 
mov dl ds:byte_467D74[esi] 
xor edx edx 
push immediate 
push edx 
push immediate 
push ebp 
mov ebp esp 
add edx eax 
lea ecx [ebp+var_28] 
push eax 
call sub_45DA00 
mov [ebp+esi+var_28] dl 
jmp 4578507 
mov eax immediate 
inc ecx 
mov eax edi 
movzx eax cx 
jmp 4578724 
mov [ebp+var_78] eax 
shr eax immediate 
xor eax edx 
push immediate 
movzx esi cx 
pop esi 
pop edi 
mov ecx [e

In [42]:
# now print nodes so we can check memaddress
!cat tmp/for_code_features/raw/graphs01/Network/malware/GetIPAddress_of_3_Hostnames_45EE80_nodes.txt

__alloca_probe_edges.txt
__alloca_probe_nodes.txt
Call_Decryption_Routine_45E320_edges.txt
Call_Decryption_Routine_45E320_nodes.txt
Check_Global_Var_460B15_edges.txt
Check_Global_Var_460B15_nodes.txt
CloseKey_and_reverttoself_45BCB0_edges.txt
CloseKey_and_reverttoself_45BCB0_nodes.txt
DecryptionRoutine01_451270_edges.txt
DecryptionRoutine01_451270_nodes.txt
DnsQuery_45AB60_edges.txt
DnsQuery_45AB60_nodes.txt
EncodeHTTPinfo_458370_edges.txt
EncodeHTTPinfo_458370_nodes.txt
GetHostbyname_and_more_45A910_edges.txt
GetHostbyname_and_more_45A910_nodes.txt
GetIPAddress_of_3_Hostnames_45EE80_edges.txt
GetIPAddress_of_3_Hostnames_45EE80_nodes.txt
GetTickCount_wrapper_edges.txt
GetTickCount_wrapper_nodes.txt
GetVersion_and_more_02_4525C0_edges.txt
GetVersion_and_more_02_4525C0_nodes.txt
GetVersion_and_more_452520_edges.txt
GetVersion_and_more_452520_nodes.txt
HOSTNAMES_Write_Hostnames_to_Stack_4581C0_edges.txt
HOSTNAMES_Write_Hostnames_to_Stack_4581C0_nodes.txt
IAT_load_WinHTTP_call_WINHTTP_45BE

In [42]:
# now print nodes so we can check memaddress
!cat tmp/for_code_features/raw/graphs01/Network/malware/GetIPAddress_of_3_Hostnames_45EE80_edges.txt

__alloca_probe_edges.txt
__alloca_probe_nodes.txt
Call_Decryption_Routine_45E320_edges.txt
Call_Decryption_Routine_45E320_nodes.txt
Check_Global_Var_460B15_edges.txt
Check_Global_Var_460B15_nodes.txt
CloseKey_and_reverttoself_45BCB0_edges.txt
CloseKey_and_reverttoself_45BCB0_nodes.txt
DecryptionRoutine01_451270_edges.txt
DecryptionRoutine01_451270_nodes.txt
DnsQuery_45AB60_edges.txt
DnsQuery_45AB60_nodes.txt
EncodeHTTPinfo_458370_edges.txt
EncodeHTTPinfo_458370_nodes.txt
GetHostbyname_and_more_45A910_edges.txt
GetHostbyname_and_more_45A910_nodes.txt
GetIPAddress_of_3_Hostnames_45EE80_edges.txt
GetIPAddress_of_3_Hostnames_45EE80_nodes.txt
GetTickCount_wrapper_edges.txt
GetTickCount_wrapper_nodes.txt
GetVersion_and_more_02_4525C0_edges.txt
GetVersion_and_more_02_4525C0_nodes.txt
GetVersion_and_more_452520_edges.txt
GetVersion_and_more_452520_nodes.txt
HOSTNAMES_Write_Hostnames_to_Stack_4581C0_edges.txt
HOSTNAMES_Write_Hostnames_to_Stack_4581C0_nodes.txt
IAT_load_WinHTTP_call_WINHTTP_45BE

#### Automated verification of GetIPAddress_of_3_Hostnames functions

In [183]:
test_code_features(dataset[103].code_feats,
                   'verification/GetIPAddress_of_3_Hostnames.txt',
                   'tmp/for_code_features/raw/graphs01/Network/malware/GetIPAddress_of_3_Hostnames_45EE80_nodes.txt',
                   'tmp/for_code_features/raw/graphs01/Network/malware/GetIPAddress_of_3_Hostnames_45EE80_edges.txt')

0x45ee80  GetIPAddress_of_3_Hostnames_45EE80 proc near 
contents is not equal:  GetIPAddress_of_3_Hostnames_45EE80 proc near |push ebp
0x45ee80  
contents is not equal:  |push ebp
0x45ee80  var_21B4 = dword ptr -21B4h 
contents is not equal:  var_21B4 = dword ptr -21B4h |push ebp
0x45ee80  var_21B0 = dword ptr -21B0h 
contents is not equal:  var_21B0 = dword ptr -21B0h |push ebp
0x45ee80  var_21AC = word ptr -21ACh 
contents is not equal:  var_21AC = word ptr -21ACh |push ebp
0x45ee80  var_21A8 = dword ptr -21A8h 
contents is not equal:  var_21A8 = dword ptr -21A8h |push ebp
0x45ee80  var_21A4 = dword ptr -21A4h 
contents is not equal:  var_21A4 = dword ptr -21A4h |push ebp
0x45ee80  var_21A0 = dword ptr -21A0h 
contents is not equal:  var_21A0 = dword ptr -21A0h |push ebp
0x45ee80  var_219C = dword ptr -219Ch 
contents is not equal:  var_219C = dword ptr -219Ch |push ebp
0x45ee80  var_2198 = dword ptr -2198h 
contents is not equal:  var_2198 = dword ptr -2198h |push ebp
0x45ee80  var_

0x45f094  loc_45F094: 
contents is not equal:  loc_45F094: |cmp cl, byte ptr [ebp+var_21AC]
0x45f094  cmp cl, byte ptr [ebp+var_21AC] 
removing previous not found memaddr
[(4583319, ' '), (4583321, ' align 10h '), (4583572, ' loc_45F094: ')]
[(4583319, ' '), (4583321, ' align 10h ')]
0x45f09a  jge short loc_45F0CE 
0x45f09c  movzx eax, al 
0x45f09f  mov edx, eax 
0x45f0a1  shr edx, 2 
0x45f0a4  shl eax, 6 
0x45f0a7  or edx, eax 
0x45f0a9  xor eax, eax 
0x45f0ab  mov al, dl 
0x45f0ad  mov [ebp+var_218C], al 
0x45f0b3  movsx esi, cl 
0x45f0b6  xor edx, edx 
0x45f0b8  mov dl, ds:byte_46953D[esi] 
0x45f0be  xor edx, eax 
0x45f0c0  mov [ebp+esi+var_28], dl 
0x45f0c4  inc cl 
0x45f0c6  mov [ebp+var_2189], cl 
0x45f0cc  jmp short loc_45F094 
0x45f0ce  
contents is not equal:  |movsx eax, cl
0x45f0ce  loc_45F0CE: 
contents is not equal:  loc_45F0CE: |movsx eax, cl
0x45f0ce  movsx eax, cl 
removing previous not found memaddr
[(4583319, ' '), (4583321, ' align 10h '), (4583630, ' '), (4583630, '

 found  0045EEB6  push edi 
found mov     [ebp+ms_exc.old_esp], esp 
 found  0045EEB7  mov [ebp+ms_exc.old_esp], esp 
found mov     [ebp+var_21A4], edx 
 found  0045EEBA  mov [ebp+var_21A4], edx 
found xor     edi, edi 
 found  0045EEC0  xor edi, edi 
found mov     [ebp+ms_exc.registration.TryLevel], edi 
 found  0045EEC2  mov [ebp+ms_exc.registration.TryLevel], edi 
found mov     byte ptr [edx], 0 
 found  0045EEC5  mov byte ptr [edx], 0 
found push    1 
 found  0045EEC8  push 1 
found push    0FFFh 
 found  0045EECA  push 0FFFh 
found lea     eax, [ebp+var_2184] 
 found  0045EECF  lea eax, [ebp+var_2184] 
found push    eax 
 found  0045EED5  push eax 
found mov     edx, 3 
 found  0045EED6  mov edx, 3 
found mov     ecx, offset dword_46987C 
 found  0045EEDB  mov ecx, offset dword_46987C 
found call    Call_Decryption_Routine_45E320 
 found  0045EEE0  call Call_Decryption_Routine_45E320 
found test    eax, eax 
 found  0045EEE5  test eax, eax 
found jnz     short loc_45EEEF 
 found 

# Verification Conclusions:

1. **Number instructions feature**: The comparison of num_instr is ok, because the lines of assembly  are greater by one because there is one align 10h instructions, which is in fact not belonging to the flow of execution of the function, so it does not appear in the document or the number of instructions.
2. **instructions in the document feature**: all instructions from the assembly listing are found in the document feature
3. **instructions in the nodes.txt**: all the instructions from the assembly listing are found in the nodes.txt list of nodes